<a href="https://colab.research.google.com/github/KaizenGirl1111/hahah/blob/master/jina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json

In [ ]:
with open("data.json","r") as file:
  documents=json.load(file)

In [ ]:
documents

[{'category': 'women',
  'description': 'YSR Pelli Kanuka Registration gives rovide financial assistance to the brides of different castes on the occasion of their marriage',
  'id': '0',
  'paths': '[ysr pelli kanuka.jpg]',
  'registration link': 'http://ysrpk.ap.gov.in',
  'state': 'Andra Pradesh'},
 {'category': 'women',
  'description': 'Under AP YSR Kapu Nestam Scheme, each women b/w age of 45 to 60 years and belonging to Kapu, Balija, Telaga, Ontari communities will get assistance. Andhra Pradesh govt. will provide financial aid of Rs. 75,000 in next 5 years (15,000 per annum) to each women beneficiary to improve their life .',
  'id': '1',
  'paths': '[AP-YSR-Kapu-Nestham-scheme-300x231.jpg]',
  'registration link': 'http://navasakam.ap.gov.in/downloads/fee_reimbursement.pdf',
  'state': 'Andra Pradesh'},
 {'category': 'poor',
  'description': 'AP YSR Aarogyasri Scheme to provide universal health coverage to all the lower and middle class families',
  'id': '2',
  'paths': '[ysr

In [ ]:
pip install cherche

In [ ]:
from cherche import retrieve

In [ ]:
retrieve.TfIdf(key="id", on=["category","description","state"],documents=documents, k=3)

TfIdf retriever
 	 key: id
 	 on: category, description, state
 	 documents: 3

In [ ]:
retriever = retrieve.TfIdf(key="id",on=["category","description","state"], documents=documents,k=3)

In [ ]:
retriever("women")

[{'id': '1', 'similarity': 0.286900784871829},
 {'id': '0', 'similarity': 0.17295354257190265}]

In [ ]:
pip install jina

In [ ]:
from docarray import Document, DocumentArray
from jina import Executor, Flow, requests

In [ ]:
class PreprocImg(Executor):
    @requests
    async def foo(self, docs: DocumentArray, **kwargs):
        for d in docs:
            (
                d.load_uri_to_image_tensor(200, 200)  # load
                .set_image_tensor_normalization()  # normalize color
                .set_image_tensor_channel_axis(
                    -1, 0
                )  # switch color axis for the PyTorch model later
            )

In [ ]:
class EmbedImg(Executor):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        import torchvision
        self.model = torchvision.models.resnet50(pretrained=True)        

    @requests
    async def foo(self, docs: DocumentArray, **kwargs):
        docs.embed(self.model)

In [ ]:
class MatchImg(Executor):
    _da = DocumentArray()

    @requests(on='/index')
    async def index(self, docs: DocumentArray, **kwargs):
        self._da.extend(docs)
        docs.clear()  # clear content to save bandwidth

    @requests(on='/search')
    async def foo(self, docs: DocumentArray, **kwargs):
        docs.match(self._da, limit=9)
        del docs[...][:, ('embedding', 'tensor')]  # save bandwidth as it is not needed

In [ ]:
f = (
    Flow(port=12345)
    .add(uses=PreprocImg)
    .add(uses=EmbedImg, replicas=3)
    .add(uses=MatchImg)
)
f.protocol = "http"

In [ ]:
index_data = DocumentArray.from_files('left/*.jpg')

In [ ]:
index_data = DocumentArray.from_files('sample/*.jpg')

In [ ]:
index_data = DocumentArray.from_files('/content/sample/*.jpg')

In [ ]:
import requests

In [ ]:
with f:
    f.post(
        '/index',
        index_data,
        show_progress=True,
        request_size=8,
    )
    r = requests.post(
"http://0.0.0.0:12345/search", headers={"Content-type": "application/json"},
json={"data": [{"uri": "/content/sample/ysr-schema.jpg"}]},).json()

Output()

╭───────────── 🎉 Flow is ready to serve! ──────────────╮
│  🔗         Protocol                          HTTP    │
│  🏠     Local access                 ]8;id=210827;HTTP://0.0.0.0:12345\0.0.0.0]8;;\]8;id=210827;HTTP://0.0.0.0:12345\:]8;;\]8;id=210827;HTTP://0.0.0.0:12345\12345]8;;\    │
│  🔒  Private network              ]8;id=993454;HTTP://172.28.0.2:12345\172.28.0.2]8;;\]8;id=993454;HTTP://172.28.0.2:12345\:]8;;\]8;id=993454;HTTP://172.28.0.2:12345\12345]8;;\    │
│  🌍   Public address           ]8;id=758478;HTTP://35.204.226.79:12345\35.204.226.79]8;;\]8;id=758478;HTTP://35.204.226.79:12345\:]8;;\]8;id=758478;HTTP://35.204.226.79:12345\12345]8;;\    │
│  💬       Swagger UI   ]8;id=183301;http://localhost:12345/docs\http://localhost:12345/docs]8;;\    │
│  📚            Redoc  ]8;id=778283;http://localhost:12345/redoc\http://localhost:12345/redoc]8;;\    │
╰───────────────────────────────────────────────────────╯

Output()

In [ ]:
import requests

In [ ]:
r

{'data': [{'adjacency': None,
   'blob': None,
   'chunks': None,
   'embedding': None,
   'evaluations': None,
   'granularity': None,
   'id': 'f3fd4630ba0f3baee6915127ca239cd7',
   'location': None,
   'matches': [{'adjacency': 1,
     'blob': None,
     'chunks': None,
     'embedding': None,
     'evaluations': None,
     'granularity': None,
     'id': 'b6c9dcdd0afe03dad3d05bd6902203dd',
     'location': None,
     'matches': None,
     'mime_type': 'image/jpeg',
     'modality': None,
     'offset': None,
     'parent_id': None,
     'scores': {'cosine': {'description': None,
       'op_name': None,
       'ref_id': None,
       'value': -1.1920928955078125e-07}},
     'tags': None,
     'tensor': None,
     'text': None,
     'uri': '/content/sample/ysr-schema.jpg',
     'weight': None},
    {'adjacency': 1,
     'blob': None,
     'chunks': None,
     'embedding': None,
     'evaluations': None,
     'granularity': None,
     'id': '6685acccf3712513364700d10db9a609',
     'loc

In [ ]:
def enhance(documents:list, **kwargs):
  data = [{"uri":f"/content/sample/{doc['paths'][0]}"} for doc in documents]
  r=requests.post(
"http://0.0.0.0:12345/search", headers={"Content-type": "application/json"},
json={"data": [{"uri": "/content/sample/ysr-schema.jpg"}]},).json()
  for document, match  in zip(documents,r["data"]):
     document["matches"]=[x["uri"] for x in match["matches"]]
  return documents

In [ ]:
retriever("women")

[{'id': '1', 'similarity': 0.286900784871829},
 {'id': '0', 'similarity': 0.17295354257190265}]

In [ ]:
enhance(retriever("women"))

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│ /usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882 in run_code  │
│                                                                                           │
│   2879 │   │   │   try:                                                                   │
│   2880 │   │   │   │   self.hooks.pre_run_code_hook()                                     │
│   2881 │   │   │   │   #rprint('Running code', repr(code_obj)) # dbg                      │
│ ❱ 2882 │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                  │
│   2883 │   │   │   finally:                                                               │
│   2884 │   │   │   │   # Reset our crash handler in place                                 │
│   2885 │   │   │   │   sys.excepthook = old_excepthook                                    │
│ <ipython-input-71-e8f685aa3b3a>:1 in <module>                                             │
│ <ipython-input-62-9a8e91321075>:2 in enhance                                              │
│ <ipython-input-62-9a8e91321075>:2 in <listcomp>                                           │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'paths'

In [ ]:
women

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│ /usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882 in run_code  │
│                                                                                           │
│   2879 │   │   │   try:                                                                   │
│   2880 │   │   │   │   self.hooks.pre_run_code_hook()                                     │
│   2881 │   │   │   │   #rprint('Running code', repr(code_obj)) # dbg                      │
│ ❱ 2882 │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                  │
│   2883 │   │   │   finally:                                                               │
│   2884 │   │   │   │   # Reset our crash handler in place                                 │
│   2885 │   │   │   │   sys.excepthook = old_excepthook                                    │
│ <ipython-input-69-342c5ee701ef>:1 in <module>                                             │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'women' is not defined